## Import needed libraries

In [2]:
import cv2 as cv
from cv2 import pyrDown, Sobel, warpAffine

## Read the consecutive frames

In [ ]:
I1 = cv.imread("")
I2 = cv.imread("")

# Pyramid construction

In [4]:
def constructImagePyramid(I1, I2):
    raise NotImplementedError

# iterative Lucas-Kanade method

In [ ]:
def pyramidalLk(I1, I2, L, w, K):
    raise NotImplementedError

## Propagate flow estimates from coarse to fine levels

In [ ]:
def Propagate_from_coarse_to_fine():
    raise NotImplementedError

## Compute the optical flow

In [ ]:
def compute_optical_flow():
    raise NotImplementedError